# Setup

## Imports

In [ ]:
import sys
import icecream
from icecream import ic
sys.path.append('./translator')

In [ ]:
import torch
import rp
import numpy as np
from translator.trainer import MUNIT_Trainer as Trainer
from translator.data import ImageFolder

In [ ]:
from source.unprojector import unproject_translations
from source.unprojector import unproject_translations_individually
from source.unprojector import combine_individual_unprojections
from source.projector import project_textures
from source.projector import colorized_scene_labels
from source.scene_reader import extract_scene_uvs_and_scene_labels
from source.color_quantizer import quantize_image

In [ ]:
from IPython.display import clear_output
from IPython.display import Video

## Other Setup

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# devuce = 'cpu'
torch.cuda.set_device(0) #Choose a free GPU

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

# Load Trainer/Data/Config

In [ ]:
# #Alphablock
# config_file       = './translator/configs/alphablock_without_ssim_256.yaml'
# image_folder_path = './datasets/alphacube/scenes/'
# # image_folder_path = './datasets/alphacube/anim_2/'
# # image_folder_path = './datasets/alphacube/anim_1/'
# image_folder_path = '/mnt/Noman/Ubuntu/CleanCode/Datasets/diff_rendering/alphabetcube_L/SyntheticData/Anim3/Renderings'
# checkpoint_folder = './translator/trained_models/outputs/alphablock_without_ssim_256/checkpoints'
# # checkpoint_folder = './translator/trained_models/outputs/alphablock_without_ssim_256/checkpoints/old_checkpoints/v0.0.7'
# # checkpoint_folder = './translator/save'
# label_values = [0,255]

In [ ]:
# #Sunkist
# config_file       = './translator/configs/config.yaml'
# image_folder_path = '/mnt/Noman/Ubuntu/CleanCode/Datasets/diff_rendering/sunkist/synthetic/RenderingsAnim'
# checkpoint_folder = './translator/trained_models/outputs/config/checkpoints'
# label_values = [0,255]

In [ ]:
# #Alphadew
# config_file       = './translator/configs/alphadew.yaml'
# image_folder_path = './datasets/alphadew/scenes'
# image_folder_path = '/home/Anonymous/CleanCode/Datasets/diff_rendering/sunkist_alphacube/synthetic/UV_Label_Exr_Anim'
# image_folder_path = '/home/Anonymous/CleanCode/Datasets/diff_rendering/sunkist_alphacube/synthetic/UV_Label_Exr_Anim_2625'
# # image_folder_path = '/home/Anonymous/CleanCode/Datasets/diff_rendering/sunkist_alphacube/synthetic/UV_Label_Exr_Mutant'
# photo_folder_path = './datasets/alphadew/photos'
# checkpoint_folder = './translator/trained_models/outputs/alphadew/checkpoints'
# label_values = [0,127,255]

In [ ]:
!ls ./translator/trained_models/outputs/

In [ ]:
VERSION_NAME='five_items__7_batch'
VERSION_NAME='five_items__shotgun' #Best one so far
# VERSION_NAME='five_items_original'
# VERSION_NAME='five_items__no_consis_but_tex'
VERSION_NAME='five_items__shotgun__variable_sizes' #Best one so far
VERSION_NAME='panorama_frontyard__var2' #Best one so far
VERSION_NAME='panorama_frontyard__var3' #Best one so far
VERSION_NAME='panorama_frontyard' #Best one so far
VERSION_NAME='panorama_frontyard' #Best one so far
VERSION_NAME='base__only_b2a' #Best one so far
# VERSION_NAME='panorama_frontyard__var3_run2' #Best one so far

In [ ]:
# #Five Items
# config_file       = './translator/configs/five_items.yaml'
# # image_folder_path = '/home/Anonymous/CleanCode/Datasets/diff_rendering/sunkist_alphacube/synthetic/UV_Label_Exr_Mutant'
# image_folder_path = './datasets/five_items/scenes'
# image_folder_path = rp.get_absolute_path('~/CleanCode/Datasets/diff_rendering/five_items/synthetic/UV_Label_Exr_Anim')
# # image_folder_path = '/mnt/Noman/Ubuntu/CleanCode/Datasets/diff_rendering/five_items/synthetic/UV_Label_Exr_Anim_2'
# photo_folder_path = './datasets/five_items/photos'

# checkpoint_folder = './translator/trained_models/outputs/%s/checkpoints'%VERSION_NAME
# config_file       = './translator/configs/%s.yaml'%VERSION_NAME

# label_values = [0,50,100,150,200,255]

In [ ]:
# #Panoramas
# image_folder_path = './datasets/panoramas/scenes'
# photo_folder_path = './datasets/panoramas/photos_frontyard'

# checkpoint_folder = './translator/trained_models/outputs/%s/checkpoints'%VERSION_NAME
# config_file       = './translator/configs/%s.yaml'%VERSION_NAME

# label_values = [0]

In [ ]:
#Three Synth Items
VERSION_NAME='three_synth_base' #Best one so far
VERSION_NAME='three_synth_base__no_texture' #Best one so far

image_folder_path = './datasets/three_synth/scenes_anim'
photo_folder_path = './datasets/three_synth/photos'

checkpoint_folder = './translator/trained_models/outputs/%s/checkpoints'%VERSION_NAME
config_file       = './translator/configs/%s.yaml'%VERSION_NAME

label_values = [0,75,150,255]

In [ ]:
config = rp.load_dyaml_file(config_file)
config = rp.DictReader(config)

In [ ]:
trainer=Trainer(config, trainable=False).to(device)
trainer.train();
# device = torch.device('cpu'); ###Why is this necessary???
# trainer=trainer.cpu()

In [ ]:
iteration=trainer.resume(checkpoint_folder)
print('Iteration:',iteration)

In [ ]:
# rp.display_image(rp.tiled_images(rp.as_numpy_images(trainer.cpu().texture_pack())))
rp.display_image(rp.tiled_images(rp.as_numpy_images(trainer.texture_pack())))

In [ ]:
aug = {}
aug["new_size_min"] = config.new_size_min_a
aug["new_size_max"] = config.new_size_max_a
aug["output_size" ] = (-1,-1) #Meaningless when skip_crop = True
# aug["output_size" ] = (320,320) #Meaningless when skip_crop = True
image_folder = ImageFolder(root=image_folder_path, precise=True, augmentation=aug)
image_folder.skip_crop = True

# Display Samples

## Test 1

In [ ]:
o=rp.random_element(image_folder)

In [ ]:
_, height, width = o.shape
icecream.ic(height,width)

In [ ]:
rp.display_image(rp.as_numpy_image(o))

In [ ]:
h=o[None].to(device)
i=trainer.sample_a2b(h)


rp.display_image(rp.as_numpy_image(h[0]))
rp.display_image(rp.as_numpy_image(i[0]))

In [ ]:
imteron=trainer.sample(h,h)[:7]
for x in imteron:
    rp.display_image(rp.as_numpy_image(x[0]))

## Test 2

In [ ]:
image_paths = rp.get_all_files(image_folder_path, sort_by='number')

In [ ]:
def scale_image(image):
    #Rescale the image to the same size it was trained on
    return rp.cv_resize_image(image, (height,width), interp='nearest')

def scale_images(images):
    return [scale_image(x) for x in images]

In [ ]:
def translate(image):
    #Input image is a UV-L Scene
    
    assert rp.is_image(image)
    
    #Rescale the image to the same size it was trained on
    image = scale_image(image)
    
    image = rp.as_rgb_image  (image)
    image = rp.as_float_image(image)
    
    image = rp.as_torch_image(image)[None] #BCHW
    image = image.to(device)
    
    with torch.no_grad():
        output = trainer.sample_a2b(image)
    output = output[0]
    output = rp.as_numpy_image(output)
    
    #Sometimes the network might change the dimensions.
    #Make sure the output is the same size as the input.
    output = rp.cv_resize_image(output, size=(height, width))
    
    return output

rp.display_image(translate(rp.load_image(rp.random_element(image_paths))))

In [ ]:
#Determinism test: Make sure it's not random!
image=rp.load_image(rp.random_element(image_paths))
trans1=translate(image)
trans2=translate(image)

print("This should be 0")
abs(trans1-trans2).max()

#This is random! Where is the randomness coming from??

# Recover Textures

In [ ]:
def get_recovered_textures(num_samples=150, recovery_resolution=750):
    #This is in a function to guarentee that all variables will be garbage-collected
    #(It can take a lot of memory)
    
    samples = (
        scale_images(
            rp.load_images(
                rp.random_batch(image_paths, min(len(image_paths),num_samples)),
                show_progress=True
            )
        )
    )
    clear_output()
    
    scene_uvs, scene_labels = (
        extract_scene_uvs_and_scene_labels(
            rp.as_torch_images(
                rp.as_numpy_array(samples)
            ),
            label_values
        )
    )
    scene_uvs[scene_uvs>1] = 1 #Due to some bug in blender's dataset generation
    
    display_eta = rp.eta(num_samples, title='Translating Samples')
    scene_trans = []
    for i,sample in enumerate(samples):
        display_eta(i)
        scene_trans.append(translate(sample))
    scene_trans = rp.as_torch_images(rp.as_numpy_array(scene_trans))
    clear_output()
        
    print("Unprojecting Textures...")
    with torch.no_grad():
        #Right now this takes a lot of ram or vram. This can be optimized.
        recovered_textures, recovered_weights = unproject_translations(scene_trans      ,
                                                                       scene_uvs,  scene_labels,
                                                                       num_labels   =len(label_values)     ,
                                                                       output_height=recovery_resolution   ,
                                                                       output_width =recovery_resolution   ,
                                                                       version = 'low memory')
    clear_output()
        
    return recovered_textures

recovered_textures = get_recovered_textures()
rp.display_image(rp.tiled_images(rp.as_numpy_images(recovered_textures)))

In [ ]:
def reproject(image):
    #Takes a UVL image and returns a reprojected image from the reconstructed texture
    assert rp.is_image(image)
    
    scene_uvs, scene_labels = (
        extract_scene_uvs_and_scene_labels(
            rp.as_torch_images(
                rp.as_numpy_array([image])
            ),
            label_values
        )
    )
        
    reprojections=project_textures(scene_uvs,scene_labels,recovered_textures)
    
    reprojection = rp.as_numpy_images(reprojections)[0]
    
    return reprojection
    
rp.display_image(reproject(rp.load_image(rp.random_element(image_paths))))

In [ ]:
def get_shadows(translation, reprojection):
    assert rp.is_image(translation)
    assert rp.is_image(reprojection)
    translation=scale_image(translation)
    reprojection=scale_image(reprojection)
    return (translation-reprojection)+.5

In [ ]:
def get_analysis_image(sample):
    assert rp.is_image(sample), 'sample is a UVL image'
    sample=scale_image(sample)
    reprojection=reproject(sample)
    translation=translate(sample)
    shadow =get_shadows(translation, reprojection)
    sample,reprojection,translation,shadow = rp.labeled_images([sample,reprojection,translation,shadow], 
                                                        ['Sample','Reprojection','Translation','Shadow'])
    return rp.grid_concatenated_images([[sample,shadow],[translation,reprojection]])

sample=rp.load_image(rp.random_element(image_paths))
rp.display_image(get_analysis_image(sample))

# Play with Reconstructed UVL's

In [ ]:

uvl=imteron[-1][0]
# # uvl=imteron[-4][0]
# rp.display_image(rp.as_numpy_image(uvl))
# def round_labels(recovered_uvl):
#     u,v,l=recovered_uvl
#     l=l[None]
#     l=quantize_image(l,torch.Tensor(label_values).to(device)[:,None]/255)
#     return torch.stack((u,v,l[0]))
    
# uvl_q=round_labels(uvl)
# pro=reproject(rp.as_numpy_image(uvl_q))
    
# rp.display_image(rp.as_numpy_image(round_labels(uvl)))
# rp.display_image(rp.as_numpy_image(abs(round_labels(uvl)-uvl))*100)
# rp.display_image(pro)

In [ ]:
scene_uvs,scene_labels=extract_scene_uvs_and_scene_labels(uvl[None],label_values)
tex=project_textures(scene_uvs,scene_labels,trainer.texture_pack())
rp.display_image(rp.as_numpy_image(tex[0]))
trans=trainer.sample_a2b(uvl[None])
rp.display_image(rp.as_numpy_image(trans[0]))

# Photos to UVL

In [ ]:
photos=rp.load_images(rp.random_batch(rp.get_all_files(photo_folder_path),10))
photo=rp.random_element(photos)
photo=scale_image(photo)
rp.display_image(photo)

In [ ]:
# t_photo=rp.as_torch_image(photo).to(device)[None]
# print(photo.shape)
# result=trainer.sample_b2a(t_photo)[0]
# tex=result[:3]
# uvl=result[3:]
# uvl[2]=quantize_image(uvl[2][None].to(device),(torch.Tensor(label_values)/255)[:,None].to(device)).double()[0]
# rp.display_image(rp.as_numpy_image(uvl))
# rp.display_image(rp.as_numpy_image(tex))

In [ ]:
rep=reproject(rp.as_numpy_image(uvl))
rp.display_image(photo)
rp.display_image(rep)

In [ ]:

scene_uvs,scene_labels=extract_scene_uvs_and_scene_labels(uvl[None],label_values)
tex=project_textures(scene_uvs,scene_labels,trainer.texture_pack())
rp.display_image(rp.as_numpy_image(tex[0]))
trans=trainer.sample_a2b(uvl[None])
rp.display_image(rp.as_numpy_image(trans[0]))
rp.display_image(photo)

In [ ]:
colors = [
    [0.5 , 0   , 0.5 ] ,
    [0.5 , 1   , 0   ] ,
    [0   , 0.5 , 0   ] ,
    [0.5 , 0.5 , 0   ] ,
    [0   , 1   , 1   ] ,
    [0   , 0   , 1   ] ,
    [0   , 0.5 , 0.5 ] ,
    [0.5 , 1   , 0.5 ] ,
    [0   , 0   , 0   ] ,
    [0.5 , 0   , 0   ] ,
    [1   , 0   , 0.5 ] ,
    [0   , 0.5 , 1   ] ,
    [1   , 0.5 , 0   ] ,
    [1   , 1   , 1   ] ,
    [0   , 1   , 0   ] ,
    [0.5 , 0.5 , 0.5 ] ,
    [0.5 , 0.5 , 1   ] ,
    [1   , 1   , 0   ] ,
    [0.5 , 1   , 1   ] ,
    [0   , 0   , 0.5 ] ,
    [1   , 0   , 0   ] ,
    [0   , 1   , 0.5 ] ,
    [0.5 , 0   , 1   ] ,
    [1   , 0.5 , 1   ] ,
    [1   , 0   , 1   ] ,
    [1   , 1   , 0.5 ] ,
    [1   , 0.5 , 0.5 ] ,
]

label_image=colorized_scene_labels(scene_labels, torch.Tensor(colors[:len(label_values)]))
rp.display_image(rp.as_numpy_image(label_image[0]))

In [ ]:
# nl=len(label_values)
# rp.display_image(rp.as_numpy_array((uvl[2]*nl).floor()/nl))

In [ ]:
# nl=len(label_values)-1
# rp.display_image(rp.as_numpy_array((uvl[2]*nl).round()/nl))

In [ ]:
# #Specific to alphadew; correction to quantizer
# label_rounded=(uvl[2]*nl).round().type(torch.int64)
# lr2=label_rounded.clone()
# lr2[lr2==1]=127
# lr2[lr2==2]=255
# # lr2=lr2/255

# ic(label_rounded[None].shape,scene_labels.shape,scene_uvs.shape)
# ic(label_rounded[None].dtype,scene_labels.dtype)

# qwe=torch.concat((scene_uvs[0],lr2[None]/255),0)
# qwe=rp.as_numpy_image(qwe)
# rp.display_image(qwe)
# rp.display_image(photo)
# rp.display_image(reproject(qwe))

# tex=project_textures(scene_uvs,label_rounded[None],trainer.texture_pack())
# rp.display_image(rp.as_numpy_image(tex[0]))
# trans=trainer.sample_a2b(uvl[None])
# rp.display_image(rp.as_numpy_image(trans[0]))

# Make a Movie

In [ ]:
display_eta = rp.eta(len(image_paths),title='Translation eta')
video_path = 'untracked/megavideo__%s__iter_%i.mp4'%(VERSION_NAME,iteration)

writer_megavideo = rp.VideoWriterMP4(video_path ,video_bitrate='max')

for i,image_path in enumerate(image_paths):
    display_eta(i)
    
    uvl_scene = rp.load_image(image_path)
    
    if not np.any(uvl_scene):
        #Blender messed up on a few frames, leaving them black...
        continue
        
    analysis_image = get_analysis_image(uvl_scene)
    writer_megavideo.write_frame(analysis_image)
    
writer_megavideo.finish()

print("Done!")